### Natural Algorithms Genetic algorithm


In [1]:
import pandas as pd
import numpy as np
import random
import csv

In [2]:
eingabe = input("Welche benchmark soll laufen? ")

with open(eingabe + ".csv") as f:
    reader = csv.reader(f)
    next(reader) # skip header
    values = []
    for row in reader:
        values.append(row)

eingabe = eingabe.split("_")

machinecount = int(eingabe[1])
jobcount = int(eingabe[2])


Welche benchmark soll laufen? benchmark1_20_300


In [3]:
population = initialize()
while terminationcondition:
    mating_pool = population.select()
    offspring = mating_pool.recombine()
    population = replace(population, offspring)
    
while terminationcondition
return best(population)

SyntaxError: invalid syntax (<ipython-input-3-f078f01070a9>, line 7)

In [ ]:
def getMax(state, values):
    for machine, job in zip(state, range(len(state))):

        machinevalues[machine] += values[job]

    return max(machinevalues)

In [ ]:
#def initialize_random(machinecount, jobcount):
# initialize empty list which represents the chromosome
jobs = []
# assign a random machine to each job    
for job in range(jobcount):
    jobs.append(random.randint(1,machinecount))
    
print(jobs)
#return jobs

In [ ]:
#def initialize_extreme(machinecount, jobcount)
# initialize empty list which represents chromosome
jobs = []
# assign the same machine to every job
machine = random.randint(1,machinecount)
for job in range(jobcount):
    jobs.append(machine)
    
#return jobs

In [ ]:
#def initialize_heuristic(machinecount, jobcount)

jobs = []
machines = []
for machine in range(machinecount):
    machines.append(0)
print(machines)
ratio = int(jobcount/machinecount)

for job in range(jobcount):
    machine = random.randint(1,machinecount)
    jobs.append(machine)
    machines[machine-1] += 1    
            
print(jobs)        
print(machines)    
    

        
        
    